### LLM Workshop -- Data Analysis using Large Language Models
Welcome to our LLM Workshop!
We hope that today you will learn some practical skills in applying Large Language Models in Data Analysis.
Enjoy and don't hesitate to ask questions!

### Prerequisites
Before we start, let's make sure that we have everything installed on our machines:

- Git;
- Pyenv;
- Our libraries (dependancies);
- Jupiter Lab;
- Ollama.

### Importing Stuff
Now that all is installed, we can start by importing any librabies we will use:

In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from langchain_ollama import OllamaLLM
from pprint import pprint
import os
from groq import Groq

### Connect and Test Model
Let's test if we can connect to the model we pulled:

In [ ]:
llm = OllamaLLM(model="llama3.2", temperature=0.05)

# answer = llm.invoke("An LLM Workshop is ...")
# pprint(answer)

In [ ]:
# Exercises:
# 1. Download and connect another model (i.e. ollama pull, ollama run)
# 2. Use it to compare the results in later exercises

### Load Data
After testing the model, it's time to load the data:

In [ ]:
df = pd.read_json("db.json")
df.head()

In [ ]:
# Exercises:
# 1. Print the number of job postings in the database
# 2. Print any other statistical metrics that would be relevant here

### Categorize Positions
With the model working and the data loaded, let's try to categorize some job positions!

In [ ]:
categories = ["Full-Stack Developer", "Front-End Developer", "Back-End Developer", "Dev Ops", "Data Engineer", "Other"]
prompt = """
    Please add an appropriate category to the following job descriptions.
    Respond with only the list of position descriptions and categories, without additional explanations.
    For example: Senior DevOps Engineer - Dev Ops, Full stack web разработчик (django rest framework и react js) - Full-Stack Developer, Senior / lead developer linux, python, c++, engleza - Back-End Developer etc. 
    Choose from the following categories: {}.
    Job Descriptions: {}.
"""
category_pattern = r"(.+) - (.+)"

def categorize_positions(llm, position_descriptions):
    llm_response = llm.invoke(prompt.format(', '.join(categories), ", ".join(position_descriptions)))
    function_response = []
    
    for line in llm_response.split("\n"):
        match = re.search(category_pattern, line)
        if match:
            position_description, category = match.groups()
            function_response.append((position_description, category))

    return function_response

# categorize_positions(llm, ["Lead Python Engineer", "Python Developer", "Python-разработчик для разработки систем"])

If privacy is not a concern, we can use existing cloud solutions.
Just don't forget to register and get your API key:

In [ ]:
api_key = "YOUR_API_KEY_HERE" # YOUR_API_KEY_HERE

def categorize_positions_groq(llm, position_descriptions):
    client = Groq(api_key=api_key)
    completion = client.chat.completions.create(
        model="llama3-groq-70b-8192-tool-use-preview",
        messages=[
            {
                "role": "user",
                "content": prompt.format(', '.join(categories), ", ".join(position_descriptions))
            }
        ],
        temperature=0.5,
        max_tokens=2048,
        top_p=0.65,
        stream=False,
        stop=None,
    )
    llm_response = completion.choices[0].message.content
    function_response = []
    
    for line in llm_response.split("\n"):
        match = re.search(category_pattern, line)
        if match:
            position_description, category = match.groups()
            function_response.append((position_description, category))

    return function_response

# categorize_positions_groq(llm, ["Lead Python Engineer", "Python Developer", "Python-разработчик для разработки систем"])

To work through our whole database of positions, run:

In [ ]:
category_df = pd.DataFrame({"Description": [], 'Category': []})
batch_size = 3

num_chunks = len(df) // batch_size + (len(df) % batch_size > 0)
batches = np.array_split(df["position_title"], num_chunks)

for i, batch in enumerate(batches, 1):
    print(f"Batch {i}..")
    new_categories = categorize_positions_groq(llm, batch.tolist())
    new_categories_df = pd.DataFrame(new_categories, columns=["Description", "Category"])
    category_df = pd.concat([category_df, new_categories_df], ignore_index=True)

category_df

### Vizualize Results
Finally we can visualize our results:

In [ ]:
def visualize_results(df, column_name):
    counts = df[column_name].value_counts()
    counts.plot(kind='bar', color='skyblue', figsize=(10, 6))
    plt.xlabel(column_name, fontsize=12)
    plt.ylabel("Number of Positions", fontsize=12)
    plt.title(f"Number of Job Positions by {column_name}", fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

visualize_results(category_df, "Category")

In [ ]:
# Exercises:
# 1. Visualize other data columns available in the database (like position_experience or position_posted)
# 2. Visualize any other statistical metrics that can be relevant for this data

In [ ]:
# Questions:
# 1. How can we handle all extra job categories?
# 2. How can we tune the model to be more focused?

### References
Congratulations for reaching here!
For those interested, we've left a couple of references you might find useful in your own projects.
Good luck!

1. [Pyenv GitHub Repo](https://github.com/pyenv/pyenv);
2. [Install Jupyter Lab](https://jupyter.org/install);
3. [Download Ollama](https://ollama.com/download);
4. [GroqCloud Playground](https://console.groq.com/playground);
5. [Docs for the OllamaLLM Class](https://python.langchain.com/api_reference/ollama/llms/langchain_ollama.llms.OllamaLLM.html).